In [202]:
initialDeclaration = []
allOutputs = []

In [203]:
def solve(layer, previousOutput, index, Adder = False, prev_non_linear = None):
  newOutput = output_var[layer] + "_" + str(index)

  if layer not in code: # Layer name's verilog not available, this condition shouldnt execute.
    print(layer, " not found")
    return []

  toAdd = [] # This is used for initial declaration.
  path = code[layer]
  changedCode = []

  # This is only for the adder
  mapping = {"vector_a" : previousOutput}
  if (prev_non_linear):
    lastChr = "abcdefghij"
    for i in range(len(prev_non_linear)):
      key = "vector_" + lastChr[i]
      mapping[key] = prev_non_linear[i]

  with open(path, 'r') as file:
    for line in file:
      curLine = line
      if "endmodule" not in curLine and "module" in curLine:
        tempLine = ""
        for ch in curLine:
          if (ch == '('):
            break
          tempLine = tempLine + ch

        tempLine = tempLine + "_" + str(index)
        toAdd.append(tempLine[7:]) # Add everything after module" ".
        curLine = tempLine + "(\n"

      if (output_var[layer] in curLine):
          curLine = curLine.replace(output_var[layer],newOutput)

      if (Adder == False):
        if(input_var[layer] in curLine and previousOutput != "None"):
            curLine = curLine.replace(input_var[layer],previousOutput)
      else:
          if "vector_" in curLine and "result_vector" not in curLine:
            check = False
            for key, value in mapping.items():
              if (key in curLine):
                curLine = curLine.replace(key,value)
                check = True

            if (check == False):
              curLine = "//" + curLine

      changedCode.append(curLine)

  if (previousOutput != "None"):
    toAdd.append(previousOutput)
  else:
    toAdd.append(input_var[layer])

  if (Adder):
    for adding in prev_non_linear:
      toAdd.append(adding)

  toAdd.append(newOutput)

  initialDeclaration.append(toAdd)

  allOutputs.append(newOutput)

  return changedCode

In [204]:
code = {}
code["Conv"] = "/content/3x3 Conv.txt"
code["Relu"] = "/content/ReLu.txt"
code["FC"] = "/content/FC.txt"
code["Pool"] = "/content/maxpooling.txt"
code["Softmax"] = "/content/softmax.txt"
code["Adder"] = "/content/Adder.txt"

In [205]:
input_var = {}
input_var["Conv"] = "input_data"
input_var["Relu"] = "din_relu"
input_var["FC"] = "input_data"
input_var["Pool"] = "in_data"
input_var["Softmax"] = "Datain"
input_var["Adder"] = ""

output_var = {}
output_var["Conv"] = "output_data"
output_var["Relu"] = "dout_relu"
output_var["FC"] = "output_data"
output_var["Pool"] = "out_data"
output_var["Softmax"] = "Dataout"
output_var["Adder"] = "result_vector"

In [206]:
params = {"vgg16" : {"parameter WIDTH" : [224, 224, 112, 112, 56, 56, 56, 28, 28, 28, 14, 14, 14],
                     "parameter HEIGHT" : [224, 224, 112, 112, 56, 56, 56, 28, 28, 28, 14, 14, 14],
                     "parameter CHANNELS" : [3, 64, 64, 128, 128, 256, 256, 256, 512, 512, 512, 512, 512],
                     "parameter FILTERS" : [64, 64, 128, 128, 256, 256, 256, 512, 512, 512, 512, 512, 512],
                     "parameter KERNEL_SIZE" : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
                     "parameter STRIDE": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     "parameter PADDING" : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
          "alexnet" : {"parameter WIDTH" : [227, 27, 13, 13, 13],
                       "parameter HEIGHT" : [227, 27, 13, 13, 13],
                       "parameter CHANNELS" : [3, 96, 256, 384, 384],
                       "parameter FILTERS" : [96, 256, 384, 384, 256],
                       "parameter KERNEL_SIZE" : [11, 5, 3, 3, 3],
                       "parameter STRIDE" : [4, 1, 1, 1, 1],
                       "parameter PADDING" : [0, 2, 1, 1, 1]},
          "resnet" : {"parameter WIDTH" : [227, 56, 56, 56, 56, 56, 28, 28, 28, 28, 14, 14, 14, 14, 7, 7, 7],
                      "parameter HEIGHT": [227, 56, 56, 56, 56, 56, 28, 28, 28, 28, 14, 14, 14, 14, 7, 7, 7],
                      "parameter CHANNELS" : [3, 64, 64, 64, 64, 64, 128, 128, 128, 128, 256, 256, 256, 256, 512, 512, 512],
                      "parameter FILTERS" : [64, 64, 64, 64, 64, 128, 128, 128, 128, 256, 256, 256, 256, 512, 512, 512, 512],
                      "parameter KERNEL_SIZE": [7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
                      "parameter STRIDE" : [2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1],
                      "parameter PADDING" : [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
          "squeezenet" : {"parameter WIDTH" : [224, 55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55, 27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27, 13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 , 13],
                          "parameter HEIGHT": [224, 55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55 ,55, 27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27 ,27, 13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 ,13 , 13],
                          "parameter CHANNELS" : [3, 96 ,96 ,96 ,96 ,96 ,96 ,96 ,96 ,96 ,96 ,96 ,96, 128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128, 256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 , 384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512],
                          "parameter FILTERS" : [96 ,96 ,96 ,96 ,96 ,96 ,96 ,96 ,96 ,96 ,96 ,96, 128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128 ,128, 256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 ,256 , 384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,384 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512 ,512, 1000],
                          "parameter KERNEL_SIZE": [7, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1],
                          "parameter STRIDE" : [2, 1, 1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1],
                          "parameter PADDING" : [0 for i in range(98)]}
          }

In [207]:
params_index = {"vgg16" : {"parameter WIDTH" : 0,
                           "parameter HEIGHT" : 0,
                           "parameter CHANNELS" : 0,
                           "parameter FILTERS" : 0,
                           "parameter KERNEL_SIZE" : 0,
                           "parameter STRIDE" : 0,
                           "parameter PADDING" : 0},
                "alexnet" : {"parameter WIDTH" : 0,
                           "parameter HEIGHT" : 0,
                           "parameter CHANNELS" : 0,
                           "parameter FILTERS" : 0,
                           "parameter KERNEL_SIZE" : 0,
                           "parameter STRIDE" : 0,
                           "parameter PADDING" : 0},
                "resnet" : {"parameter WIDTH" : 0,
                           "parameter HEIGHT" : 0,
                           "parameter CHANNELS" : 0,
                           "parameter FILTERS" : 0,
                           "parameter KERNEL_SIZE" : 0,
                           "parameter STRIDE" : 0,
                           "parameter PADDING" : 0},
                "squeezenet" : {"parameter WIDTH" : 0,
                                "parameter HEIGHT" : 0,
                                "parameter CHANNELS" : 0,
                                "parameter FILTERS" : 0,
                                "parameter KERNEL_SIZE" : 0,
                                "parameter STRIDE" : 0,
                                "parameter PADDING" : 0}
                }
temp = 0

In [208]:
def change(lines, model): # Parameters change
   newCode = []
   for j in range(len(lines)):
    paramFound = False
    for key in params[model]:
       value = params[model][key]
       if(key in lines[j]):
          paramFound = True
          s = ''
          for k in range(len(lines[j])):
            s+=lines[j][k]
            if(lines[j][k] == "="):
              s += " "
              index = params_index[model][key]
              s+=str(value[index])
              s += ";\n"
              params_index[model][key] = index + 1
              break
          newCode.append(s)
    if (paramFound == False):
      newCode.append(lines[j])
   return newCode

In [209]:
finalCode = []

In [210]:
skip = True
previousOutput = "None"
idx_module = 1
model = "squeezenet"
modelPath = model + ".txt"

with open(modelPath, 'r') as file:

    for line in file:
      line = line.strip()
      if (len(line) == 0 or skip):
        skip = False
        continue

      words = []
      curWord = ""
      for chr in line:
        if (chr == ","):
          words.append(curWord)
          curWord = ""
        else:
          curWord = curWord + chr

      inputsCount = int(words[0])
      idx = 1
      inputs = []
      while (inputsCount):
        inputsCount = inputsCount - 1
        if (words[idx] != "None"):
          inputs.append(allOutputs[int(words[idx])])
        else:
          inputs.append("None")
        idx = idx + 1

      layer = words[idx]

      previousOutput = inputs[0] # Make sure in the text file, first num is the previous output
      inputs = inputs[1:]

      if layer == "Adder":
        lines = solve(layer, previousOutput, idx_module, True, inputs)
      else:
        lines = solve(layer, previousOutput, idx_module)

      idx_module = idx_module + 1

      if (layer == "Conv"): ## Check
        lines = change(lines, model)

      finalCode.append(lines)

In [211]:
first = "module top(input inp, input clk, input rst, output out);\n"
space = "    "
second = space + "wire output_data;\n"
third = space + "wire dout_relu;\n"
fourth = space + "wire Start;\n"
fifth = space + "wire [2:0] N;\n"

In [212]:
output_file = model + "_output_file.txt"

In [213]:
with open(output_file, 'w') as output_file:
  output_file.write(first)
  output_file.write(second)
  output_file.write(third)
  output_file.write(fourth)
  output_file.write(fifth)

  for i in range(len(initialDeclaration)):
    string = initialDeclaration[i][0] + " uut"
    if (i):
      string = string + str(i)

    layer = initialDeclaration[i][0][:4]

    if (layer == "vect"): # Adder module name - vector_addition
      declarations = "("
      for j in range(1, len(initialDeclaration[i]) - 1):
        declarations = declarations + ".input_data" + str(j) + "(" + initialDeclaration[i][j] + "), "
      string = space + string + declarations + ".output_data(" + initialDeclaration[i][-1] + "));\n"
    else:
      if (layer == "Conv"):
        string = string + "(.clk(clk),.rst(rst),.input_data("
      else:
        string = string + "(.input_data("

      string = space + string + initialDeclaration[i][1] + "), .output_data(" + initialDeclaration[i][2] + "));\n"

    output_file.write(string)

  output_file.write("endmodule\n")

  for i in finalCode:
    for j in i:
      output_file.write(j)

In [214]:
for i in initialDeclaration:
  print(i)

['Conv2D _1', 'input_data', 'output_data_1']
['maxpool_2', 'output_data_1', 'out_data_2']
['Conv2D _3', 'out_data_2', 'output_data_3']
['Conv2D _4', 'out_data_2', 'output_data_4']
['Conv2D _5', 'out_data_2', 'output_data_5']
['Conv2D _6', 'out_data_2', 'output_data_6']
['vectorAdder_7', 'output_data_6', 'output_data_3', 'output_data_4', 'output_data_5', 'result_vector_7']
['Conv2D _8', 'result_vector_7', 'output_data_8']
['Conv2D _9', 'result_vector_7', 'output_data_9']
['Conv2D _10', 'result_vector_7', 'output_data_10']
['Conv2D _11', 'result_vector_7', 'output_data_11']
['Conv2D _12', 'result_vector_7', 'output_data_12']
['Conv2D _13', 'result_vector_7', 'output_data_13']
['Conv2D _14', 'result_vector_7', 'output_data_14']
['Conv2D _15', 'result_vector_7', 'output_data_15']
['vectorAdder_16', 'output_data_15', 'output_data_8', 'output_data_9', 'output_data_10', 'output_data_11', 'output_data_12', 'output_data_13', 'output_data_14', 'result_vector_16']
['Conv2D _17', 'result_vector_16